##### Importing and constant defining section:


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
# to set current working directory coherent_neutrino_scattering/ (not coherent_neutrino_scattering/vis/)
if os.getcwd()[-4:] == '/vis':
    os.chdir('../')
%matplotlib inline

from src.config import config
import src.coherent_neutrino_nucleus as cn
import src.phys_constants as p_const
from src.SpectrumInterpolator import SpectrumInterpolator

Ar_N = 21
Ar_Z = 18
Ar_M = 38.964313 * p_const.ATOMIC_MASS_UNIT # MeV

### Sample Reactor Antineutrino Flux and Spectrum

plots

formula for total flow

### Kinetic energy spectrum of recoil nuclei

$$ \frac{dN}{dT} = \int dE_{\nu} \frac{dF_{\nu}}{dE_{\nu}} \frac{d\sigma(E_{\nu})}{dT} $$

Initial def:

In [4]:
nu_energy_range = np.linspace(1, 7, 100) # MeV
nucleus_max_kinetic = cn.max_kinetic_recoil_nucleus(Ar_M, nu_energy_range[nu_energy_range.size - 1]) # MeV
nucleus_kinetic_range = np.linspace(0, nucleus_max_kinetic, 100)

Loading and interpolating reactor neutrino spectrum:

In [6]:
data_path = os.path.abspath(config['DATA_PATH'] + 'mock_reactor_neutrino_spectrum.csv')
interpolator = SpectrumInterpolator(data_path, 'E', 'dN_dE')

nu_energy_spectrum = []
for i in range(nu_energy_range.size):
    probability_density = interpolator.interpolate(nu_energy_range[i])
    nu_energy_spectrum.append(probability_density)
    
nu_energy_spectrum = np.array(nu_energy_spectrum)

Producing recoil nuclei spectrum (integration):

In [ ]:
en_increment = nu_energy_spectrum[1] - nu_energy_spectrum[0]
nucleus_kinetic_spectrum = []
for nucleus_kinetic in nucleus_kinetic_range:
    #min nu energy to integrate
    min_nu_energy = cn.min_energy_neutrino_produce_recoil_nucleus(Ar_M, nucleus_kinetic) 
    #index in nu_energy_range
    first_index = np.argmax(nu_energy_range >= min_nu_energy)
    
    probability_density = 0
    for i in range(first_index, nu_energy_range.size):
        diff_cs = cn.differential_cross_section_for_nucleus_kinetic(Ar_Z, Ar_N, Ar_M, nu_energy_range[i], nucleus_kinetic)
        probability_density += en_increment * nu_energy_spectrum[i] * diff_cs
        
    nucleus_kinetic_spectrum.append(probability_density)

nucleus_kinetic_spectrum = np.array(nucleus_kinetic_spectrum)